In [1]:
# 오토인코더  = 자연과학... 불량품 판정  양품이 대수고.. 불량데이터는 극히적어..

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras import metrics
from tensorflow.keras.datasets import mnist

In [3]:
# 주요 변수
batch_size = 100
original_dim = 784
latent_dim = 2
intermediate_dim = 256
epochs = 50
epsilon_std = 1.0

In [4]:
# 셈플링 함수
def sampling(args:tuple):
  z_mean, z_log_var = args
  epsilon = K.random_normal(shape = (K.shape(z_mean)[0], latent_dim), mean=0. , stddev = epsilon_std  )
  return z_mean + K.exp(z_log_var / 2) * epsilon

In [5]:
from numpy.core.fromnumeric import shape
# 인코더 정의
x = Input(shape = (original_dim,))
# 중간층
h = Dense(intermediate_dim,activation='relu')(x)
# 잠복공간의 평균을 정의
z_mean = Dense(latent_dim)(h)
# 잠복공간의 로그분산을 정의
z_log_var = Dense(latent_dim)(h)
# 역전파를이용... 텐서플로의 벡엔드를 이용한다.
z = Lambda(sampling, output_shape = (latent_dim,))([z_mean, z_log_var])
# 케라스 모델로 인코더를 정의
encoder = Model(x, [z_mean, z_log_var, z])
encoder.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 784)]        0           []                               
                                                                                                  
 dense (Dense)                  (None, 256)          200960      ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 2)            514         ['dense[0][0]']                  
                                                                                                  
 dense_2 (Dense)                (None, 2)            514         ['dense[0][0]']                  
                                                                                              

In [6]:
# 디코더 정의
input_decoder = Input(shape=(latent_dim,))
h = Dense(intermediate_dim, activation='relu')(input_decoder)
x = Dense(original_dim, activation='sigmoid')(h)
decoder = Model(input_decoder,x)
decoder.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 2)]               0         
                                                                 
 dense_3 (Dense)             (None, 256)               768       
                                                                 
 dense_4 (Dense)             (None, 784)               201488    
                                                                 
Total params: 202,256
Trainable params: 202,256
Non-trainable params: 0
_________________________________________________________________
